# Charles de Kock - 26023830 - ML441 Assignment 2
## Decision Trees

In [86]:
import pandas as pd
import numpy as np
import os
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import RobustScaler
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTETomek
from sklearn.model_selection import StratifiedKFold
from tqdm import tqdm
from sklearn.metrics import f1_score, matthews_corrcoef
import json
import itertools
from sklearn.tree import DecisionTreeClassifier
from collections import defaultdict

In [126]:
data_og = pd.read_csv(os.path.dirname(os.getcwd()) + '/forestCover.csv', na_values='?')

### Preprocess

In [ ]:
def init_dt_preprocessing(df: pd.DataFrame):

    df = df.drop(columns=['Aspect', 'Inclination', 'Water_Level', 'Observation_ID'])
    df['Soil_Type1'] = df['Soil_Type1'].map({'positive': 1, 'negative': 0}).astype(np.int8)

    imputer = SimpleImputer(strategy="median")
    df = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)

    return df

In [128]:
data_clean = init_dt_preprocessing(data_og)

In [129]:
def balancing_transform(df: pd.DataFrame, target, strategy_over, strategy_under, r=None):
    X = df.drop(columns=[target])
    y = df[target]

    under = RandomUnderSampler(sampling_strategy=strategy_under, random_state=r)
    X_under, y_under = under.fit_resample(X, y)

    smote = SMOTE(sampling_strategy=strategy_over, random_state=r+1)
    #smote_tomek = SMOTETomek(sampling_strategy=strategy_over, random_state=r) #consider using later, takes long tho
    X_res, y_res = smote.fit_resample(X_under, y_under)

    df_balanced = pd.DataFrame(X_res, columns=X.columns)
    df_balanced[target] = y_res

    return df_balanced

def balancing_transform_lean(X, y, strategy_over, strategy_under, r=None):

    under = RandomUnderSampler(sampling_strategy=strategy_under, random_state=r)
    X_under, y_under = under.fit_resample(X, y)

    smote = SMOTE(sampling_strategy=strategy_over, random_state=r)
    X_res, y_res = smote.fit_resample(X_under, y_under)

    return X_res, y_res

In [130]:
X_clean = data_clean.drop(columns=['Cover_Type'])
y_clean = data_clean['Cover_Type']

 ### Cross fold setup

In [132]:
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=None)

folds = []

for train_index, val_index in skf.split(X_clean, y_clean):
    X_train, X_val = X_clean.iloc[train_index], X_clean.iloc[val_index]
    y_train, y_val = y_clean.iloc[train_index], y_clean.iloc[val_index]
    folds.append({'Xt': X_train, 'Xv': X_val, 'yt': y_train, 'yv': y_val})


In [133]:
def oversample_fold(fold, strategies):
    X, y = balancing_transform_lean(fold['Xt'], fold['yt'], strategies[1], strategies[0])
    return {'Xt': X, 'Xv': fold['Xv'], 'yt': y, 'yv': fold['yv']}

In [134]:
strat_A = [{1: 50000, 2: 50000}, {3: 35000, 4: 20000, 5: 20000, 6: 30000, 7: 30000}]

for i in tqdm(range(len(folds))):
    folds[i] = oversample_fold(folds[i], strategies=strat_A)

100%|██████████| 10/10 [00:40<00:00,  4.08s/it]


### Tuning

In [135]:
def eval_model(fold, fold_nr, param_id, params):
    model = DecisionTreeClassifier(criterion=params['criterion'],
                                   max_depth=params['max_depth'], 
                                   min_samples_leaf=params['min_samples_leaf'], 
                                   class_weight=params['class_weight'])
    
    model.fit(fold['Xt'], fold['yt'])
    y_pred = model.predict(fold['Xv'])
    f1_w = f1_score(fold['yv'], y_pred, average='weighted')
    f1_m = f1_score(fold['yv'], y_pred, average='macro')
    mcc = matthews_corrcoef(fold['yv'], y_pred)
    
    return {'fold_nr': fold_nr, 'param_id': param_id,'params': params, 'mcc': mcc, 'f1_weigthed': f1_w, 'f1_macro': f1_m}

In [136]:
def generate_param_dicts(criteria, depths, min_samples_leaf, weight):
    param_dicts = []
    for c, d, ms, w in itertools.product(criteria, depths, min_samples_leaf, weight):
            params = {
                'criterion': c,
                'max_depth': d, 
                'min_samples_leaf': ms, 
                'class_weight': w
            }
            param_dicts.append(params)
    return param_dicts

In [137]:
def grid_search(param_grid, folds):
    out = []
    idx = 0
    for i in tqdm(range(len(param_grid))):
        for j in range(len(folds)):
            out.append(eval_model(folds[j], j, i, param_grid[i]))
            #print("Params:", i, '\nFold:', j, '\n')
    return out

In [138]:
def save_results(results, file):
    with open(file, "w") as f:
        for d in results:
            f.write(json.dumps(d) + "\n")

def load_results(file):
    loaded_results = []
    with open(file, "r") as f:
        for line in f:
            loaded_results.append(json.loads(line))
    return loaded_results

In [143]:
param_grid = generate_param_dicts(['gini', 'entropy'], np.linspace(20, 50, 12).astype(int), [1, 0.0002], [None, 'balanced'])

In [144]:
results_fine = grid_search(param_grid, folds)
save_results(results_fine, "eval_fine_dt.txt")

100%|██████████| 96/96 [1:29:22<00:00, 55.85s/it]


TypeError: Object of type int64 is not JSON serializable

In [145]:
for result in results_fine:
    result['params']['max_depth'] = int(result['params']['max_depth'])

In [146]:
save_results(results_fine, "eval_fine_dt_backup.txt")

### Evaluation

In [87]:
def avg_results(data):
    grouped_data = defaultdict(list)
    for d in data:
        grouped_data[d['param_id']].append(d)

    summary_list = []
    for param_id, group in grouped_data.items():
        mcc_scores = [d['mcc'] for d in group]
        f1_weigthed_scores = [d['f1_weigthed'] for d in group]
        f1_macro_scores = [d['f1_macro'] for d in group]

        summary = {
            'param_id': param_id,
            'params': group[0]['params'],
            'mean_mcc': np.mean(mcc_scores),
            'std_mcc': np.std(mcc_scores),
            'mean_f1_weigthed': np.mean(f1_weigthed_scores),
            'std_f1_weigthed': np.std(f1_weigthed_scores),
            'mean_f1_macro': np.mean(f1_macro_scores),
            'std_f1_macro': np.std(f1_macro_scores)
        }
        summary_list.append(summary)

    return summary_list

In [94]:
def sort_results(results, key, show=False):
    out = sorted(results, key=lambda x: x['mean_' + key], reverse=True)
    if show == True:
        for r in out:
            print(f"ID: {r['param_id']} Parameters: {r['params']} -> {key}: {r['mean_' + key]} ± {r['std_' + key]}")
    return out

In [100]:
sorted_fine_evals_mcc = sort_results(avg_results(results_fine), 'mcc', show=False)
sorted_fine_evals_f1w = sort_results(avg_results(results_fine), 'f1_weigthed', show=False)
sorted_fine_evals_f1m = sort_results(avg_results(results_fine), 'f1_macro', show=False)

In [109]:
def combine_evals(eval1, eval2, eval3):
    out = defaultdict(int)
    for i in range(len(eval1)):
        out[str(eval1[i]['param_id'])] += (i + 1)
    for i in range(len(eval2)):
        out[str(eval2[i]['param_id'])] += (i + 1)
    for i in range(len(eval3)):
        out[str(eval3[i]['param_id'])] += (i + 1)
    return out

In [124]:
combined_ranks = combine_evals(sorted_fine_evals_mcc, sorted_fine_evals_f1w, sorted_fine_evals_f1m)
sorted(combined_ranks.items(), key=lambda item: item[1], reverse=False)

[('183', 8),
 ('182', 9),
 ('169', 10),
 ('168', 10),
 ('211', 11),
 ('197', 16),
 ('210', 21),
 ('196', 25),
 ('155', 26),
 ('154', 29),
 ('141', 34),
 ('140', 35),
 ('56', 41),
 ('84', 42),
 ('70', 46),
 ('98', 50),
 ('126', 51),
 ('42', 53),
 ('28', 57),
 ('127', 57),
 ('99', 62),
 ('85', 66),
 ('71', 69),
 ('57', 72),
 ('43', 76),
 ('14', 77),
 ('29', 82),
 ('112', 83),
 ('113', 87),
 ('15', 91),
 ('0', 92),
 ('1', 96),
 ('212', 99),
 ('198', 103),
 ('184', 107),
 ('142', 109),
 ('170', 109),
 ('128', 114),
 ('156', 115),
 ('100', 123),
 ('72', 126),
 ('58', 129),
 ('44', 130),
 ('86', 130),
 ('30', 133),
 ('114', 134),
 ('16', 139),
 ('2', 144),
 ('143', 147),
 ('171', 150),
 ('213', 153),
 ('199', 157),
 ('185', 158),
 ('157', 162),
 ('129', 165),
 ('115', 168),
 ('87', 172),
 ('101', 175),
 ('73', 175),
 ('59', 180),
 ('45', 183),
 ('31', 186),
 ('17', 189),
 ('3', 200),
 ('18', 203),
 ('32', 206),
 ('88', 209),
 ('144', 210),
 ('46', 212),
 ('200', 214),
 ('74', 215),
 ('186', 

In [140]:
def find_ID(results , id):
    return [d for d in results if d.get('param_id') == id]

In [141]:
best = find_ID(results_fine, 183)

In [142]:
best

[{'fold_nr': 0,
  'param_id': 183,
  'params': {'criterion': 'entropy',
   'max_depth': 37,
   'min_samples_leaf': 1,
   'class_weight': 'balanced'},
  'mcc': 0.8325450423298141,
  'f1_weigthed': 0.8944733353057686,
  'f1_macro': 0.8644708657374887},
 {'fold_nr': 1,
  'param_id': 183,
  'params': {'criterion': 'entropy',
   'max_depth': 37,
   'min_samples_leaf': 1,
   'class_weight': 'balanced'},
  'mcc': 0.829249286534213,
  'f1_weigthed': 0.8922611004328261,
  'f1_macro': 0.8665297012516382},
 {'fold_nr': 2,
  'param_id': 183,
  'params': {'criterion': 'entropy',
   'max_depth': 37,
   'min_samples_leaf': 1,
   'class_weight': 'balanced'},
  'mcc': 0.832406632431436,
  'f1_weigthed': 0.894402657715297,
  'f1_macro': 0.8631082114259925},
 {'fold_nr': 3,
  'param_id': 183,
  'params': {'criterion': 'entropy',
   'max_depth': 37,
   'min_samples_leaf': 1,
   'class_weight': 'balanced'},
  'mcc': 0.8309053420215569,
  'f1_weigthed': 0.8933544982179393,
  'f1_macro': 0.8668426779934579},